In [112]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [113]:
response = requests.get('https://www.epicurious.com/search/cucumbers')
doc = BeautifulSoup(response.text)

In [114]:
rows =[]
url = ''
for page_num in range(1,11): 
    url = f'https://www.epicurious.com/search/cucumbers?{page_num}'
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    cucumbers = doc.find_all('article')
    for cucumber in cucumbers:
        row = {}
        row['tag'] = cucumber['class'][0]
        a_link = cucumber.find('a', class_='view-complete-item',itemprop = 'url')
        try:
            row['title'] = a_link['title']
        except:
            pass
        try:
            row['summary'] = cucumber.find('header',class_= 'summary').find('p',class_='dek').text
        except:
            pass
        try:
            row['rating'] = cucumber.find('span',itemprop = 'ratingValue').text
        except:
            pass
        try:
            row['would make again'] = cucumber.find('dd',class_ ='make-again-percentage').text
        except:
            pass
        try:
            row['url'] = f"https://www.epicurious.com{a_link['href']}"
        except:
            pass
        rows.append(row)
      



In [116]:
df = pd.DataFrame(rows)
df

,rating,summary,tag,title,url,would make again
0,4,These quick pickles have just the right amount...,recipe-content-card,Spicy Lightly Pickled Cucumbers,https://www.epicurious.com/recipes/food/views/...,100%
1,4,Think of this as an all-purpose garlic sauce. ...,recipe-content-card,Cucumbers with Ajo Blanco Sauce,https://www.epicurious.com/recipes/food/views/...,100%
2,2,Beef tenderloin is precious enough to baby on ...,recipe-content-card,Cold Beef Tenderloin with Tomatoes and Cucumbers,https://www.epicurious.com/recipes/food/views/...,0%
3,3,Take your cast-iron pan to the grill for this ...,recipe-content-card,Fried Fish Sandwiches with Cucumbers and Tarta...,https://www.epicurious.com/recipes/food/views/...,100%
4,3,We call for semi-pearled grains because they c...,recipe-content-card,Grain Salad with Tomatoes and Cucumbers,https://www.epicurious.com/recipes/food/views/...,100%
5,2,Go hard when charring the cabbage wedges—to th...,recipe-content-card,Charred Cabbage with Goat Cheese Raita and Cuc...,https://www.epicurious.com/recipes/food/views/...,0%
6,0,A ginger and coconut aminos dressing brings to...,recipe-content-card,Cucumber-Seaweed Salad,https://www.epicurious.com/recipes/food/views/...,0%
7,3.5,Quickly caramelized pineapple and sautéed shri...,recipe-content-card,Pineapple Shrimp Noodle Bowls,https://www.epicurious.com/recipes/food/views/...,73%
8,3.5,"The shrimp cooks in minutes, so while it’s mar...",recipe-content-card,Sambal Shrimp Lettuce Wraps,https://www.epicurious.com/recipes/food/views/...,50%
9,0,"If you have ever peeled, seeded, and sliced cu...",recipe-content-card,Smashed Cucumber Salad with Lemon and Celery Salt,https://www.epicurious.com/recipes/food/views/...,0%


In [117]:
df.to_csv("epicurious_cucumbers.csv", index=False)

### Part 2

In [118]:
df = pd.read_csv("epicurious_cucumbers.csv")
df = df[df['tag']=='recipe-content-card']


https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers
https://www.epicurious.com/recipes/food/views/cucumbers-with-ajo-blanco-sauce
https://www.epicurious.com/recipes/food/views/cold-beef-tenderloin-with-tomatoes-and-cucumbers
https://www.epicurious.com/recipes/food/views/fried-fish-sandwiches-with-cucumbers-and-tartar-sauce
https://www.epicurious.com/recipes/food/views/farro-spelt-grain-salad-with-tomatoes-and-cucumbers
https://www.epicurious.com/recipes/food/views/grilled-charred-cabbage-with-goat-cheese-raita-and-cucumbers
https://www.epicurious.com/recipes/food/views/cucumber-seaweed-salad
https://www.epicurious.com/recipes/food/views/pineapple-shrimp-noodle-bowls
https://www.epicurious.com/recipes/food/views/sambal-shrimp-lettuce-wraps
https://www.epicurious.com/recipes/food/views/smashed-cucumber-salad-with-lemon-and-celery-salt
https://www.epicurious.com/recipes/food/views/thai-style-squid-and-cucumber-salad
https://www.epicurious.com/recipes/food/views/s

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
16     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
30     None
31     None
32     None
       ... 
154    None
155    None
156    None
157    None
158    None
159    None
160    None
161    None
162    None
163    None
164    None
165    None
166    None
168    None
171    None
172    None
173    None
174    None
175    None
176    None
177    None
178    None
179    None
180    None
181    None
182    None
183    None
184    None
185    None
187    None
Length: 160, dtype: object